1. Constrained Features

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree

from tqdm import tqdm

# Load BurST-ADMA dataset (Assuming CSV format)
file_path = "imported_data/BurST-ADMA_v0.1.csv"  # Update with actual path
data = pd.read_csv(file_path)

# Extract relevant fields
vehicles = data[['id', 'timestep', 'x', 'y', 'speed', 'heading','acceleration']]

# Define sensor range (meters)
SENSOR_RANGE = 50

# Function to simulate CPM detection
def generate_cpm(vehicles, sensor_range=SENSOR_RANGE):
    cpm_data = []
    timestamps = vehicles['timestep'].unique()
    
    for t in tqdm(timestamps):
        snapshot = vehicles[vehicles['timestep'] == t]
        positions = snapshot[['x', 'y']].values
        ids = snapshot['id'].values
        kd_tree = KDTree(positions)
        
        for i, (vid, pos) in enumerate(zip(ids, positions)):
            # Find nearest neighbors within sensor range
            indices = kd_tree.query_ball_point(pos, sensor_range)
            
            detected_objects = []
            for idx in indices:
                if idx != i:  # Exclude self
                    detected_objects.append({
                        'detected_id': ids[idx],
                        'x_detected': positions[idx][0] + np.random.normal(0, 1),  # Add noise
                        'y_detected': positions[idx][1] + np.random.normal(0, 1)
                    })
            
            # Create CPM entry
            cpm_data.append({
                'id': vid,
                'timestep': t,
                'detected_objects': detected_objects
            })
    
    return pd.DataFrame(cpm_data)

# Generate CPM dataset
cpm_dataset = generate_cpm(vehicles)

# Save enhanced dataset
cpm_dataset.to_csv("burst_adma_with_cpm.csv", index=False)
print("CPM-enhanced dataset saved as burst_adma_with_cpm.csv")


100%|██████████| 1000/1000 [02:02<00:00,  8.13it/s]


CPM-enhanced dataset saved as burst_adma_with_cpm.csv


2. +Extended Features

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree


from tqdm import tqdm

# Load BurST-ADMA dataset (Assuming CSV format)
file_path = "imported_data/BurST-ADMA_v0.1.csv"  # Update with actual path
data = pd.read_csv(file_path)

# Extract relevant fields
vehicles = data[['id', 'timestep', 'x', 'y', 'speed', 'heading','acceleration']]

# Define sensor range (meters)
SENSOR_RANGE = 50
SENSOR_TYPE = "LiDAR"

# Function to simulate CPM detection
def generate_cpm(vehicles, sensor_range=SENSOR_RANGE):
    cpm_data = []
    timestamps = vehicles['timestep'].unique()
    
    for t in tqdm(timestamps):
        snapshot = vehicles[vehicles['timestep'] == t]
        positions = snapshot[['x', 'y']].values
        ids = snapshot['id'].values
        velocities = snapshot['speed'].values
        headings = snapshot['heading'].values
        accelerations = snapshot['acceleration'].values
        kd_tree = KDTree(positions)
        
        for i, (vid, pos, vel, head) in enumerate(zip(ids, positions, velocities, headings,accelerations)):
            # Find nearest neighbors within sensor range
            indices = kd_tree.query_ball_point(pos, sensor_range)
            
            detected_objects = []
            for idx in indices:
                if idx != i:  # Exclude self
                    detected_objects.append({
                        'detected_id': ids[idx],
                        'x_detected': positions[idx][0] + np.random.normal(0, 1),  # Add noise
                        'y_detected': positions[idx][1] + np.random.normal(0, 1),
                        'speed_detected': velocities[idx] + np.random.normal(0, 0.5),
                        'heading_detected': headings[idx] + np.random.normal(0, 1),
                        'acceleration_detected': accelerations[idx] + np.random.normal(0, 1),
                        'object_type': 'vehicle',  # Assuming vehicles for now
                        'detection_confidence': np.random.uniform(0.8, 1.0)  # High confidence
                    })
            
            # Create CPM entry
            cpm_data.append({
                'cpm_id': f"CPM_{vid}_{t}",
                'id': vid,
                'timestep': t,
                'sensor_id': f"Sensor_{vid}",
                'sensor_type': SENSOR_TYPE,
                'weather_conditions': 'clear',  # Placeholder
                'visibility_range': sensor_range,
                'detected_objects': detected_objects
            })
    
    return pd.DataFrame(cpm_data)

# Generate CPM dataset
cpm_dataset = generate_cpm(vehicles)

# Save enhanced dataset
cpm_dataset.to_csv("burst_adma_with_cpm_enhanced.csv", index=False)
print("CPM-enhanced dataset saved as burst_adma_with_cpm.csv")


100%|██████████| 1000/1000 [04:45<00:00,  3.51it/s]


CPM-enhanced dataset saved as burst_adma_with_cpm.csv


3. +Environment Features

In [2]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree

from tqdm import tqdm

# Load BurST-ADMA dataset (Assuming CSV format)
file_path = "imported_data/BurST-ADMA_v0.1.csv"  # Update with actual path
data = pd.read_csv(file_path)

# Extract relevant fields
vehicles = data[['id', 'timestep', 'x', 'y', 'speed', 'heading','acceleration']]

# Define sensor range (meters)
SENSOR_RANGE = 50
SENSOR_TYPE = "LiDAR"

# Function to simulate CPM detection
def generate_cpm(vehicles, sensor_range=SENSOR_RANGE):
    cpm_data = []
    timestamps = vehicles['timestep'].unique()
    
    # Simulate environmental conditions
    weather_conditions = np.random.choice(['clear', 'foggy', 'rainy', 'snowy'], size=len(timestamps))
    visibility_ranges = {
        'clear': 100,
        'foggy': 30,
        'rainy': 50,
        'snowy': 40
    }
    
    for idx, t in tqdm(enumerate(timestamps)):
        snapshot = vehicles[vehicles['timestep'] == t]
        positions = snapshot[['x', 'y']].values
        vehicle_ids = snapshot['id'].values
        velocities = snapshot['speed'].values
        headings = snapshot['heading'].values
        accelerations = snapshot['acceleration'].values
        kd_tree = KDTree(positions)
        
        current_weather = weather_conditions[idx]
        visibility_range = visibility_ranges[current_weather]
        
        for i, (vid, pos, vel, head,acceleration) in enumerate(zip(vehicle_ids, positions, velocities, headings,accelerations)):
            # Find nearest neighbors within sensor range
            indices = kd_tree.query_ball_point(pos, min(sensor_range, visibility_range))
            
            detected_objects = []
            for idx in indices:
                if idx != i:  # Exclude self
                    detected_objects.append({
                        'detected_vehicle_id': vehicle_ids[idx],
                        'x_detected': positions[idx][0] + np.random.normal(0, 1),  # Add noise
                        'y_detected': positions[idx][1] + np.random.normal(0, 1),
                        'velocity_detected': velocities[idx] + np.random.normal(0, 0.5),
                        'heading_detected': headings[idx] + np.random.normal(0, 1),
                        'acceleration_detected': accelerations[idx] + np.random.normal(0, 1),
                        'object_type': 'vehicle',  # Assuming vehicles for now
                        'detection_confidence': np.random.uniform(0.6, 1.0)  # Varies with weather
                    })
            
            # Create CPM entry
            cpm_data.append({
                'cpm_id': f"CPM_{vid}_{t}",
                'id': vid,
                'timestep': t,
                'sensor_id': f"Sensor_{vid}",
                'sensor_type': SENSOR_TYPE,
                'weather_conditions': current_weather,
                'visibility_range': visibility_range,
                'detected_objects': detected_objects
            })
    
    return pd.DataFrame(cpm_data)

# Generate CPM dataset
cpm_dataset = generate_cpm(vehicles)

# Save enhanced dataset
cpm_dataset.to_csv("burst_adma_with_cpm_and_env.csv", index=False)
print("CPM-enhanced dataset saved as burst_adma_with_cpm_and_env.csv")


1000it [06:04,  2.74it/s]


CPM-enhanced dataset saved as burst_adma_with_cpm_and_env.csv


4. +Multiple Sensors

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree

from tqdm import tqdm

# Load BurST-ADMA dataset (Assuming CSV format)
file_path = "imported_data/BurST-ADMA_v0.1.csv"  # Update with actual path
data = pd.read_csv(file_path)

# Extract relevant fields
vehicles = data[['id', 'timestep', 'x', 'y', 'speed', 'heading','acceleration']]

# Define sensor range (meters)
SENSOR_TYPES = ["LiDAR", "Camera", "Radar"]
SENSOR_RANGES = {"LiDAR": 50, "Camera": 40, "Radar": 80}
MAX_TS=200
# Function to extract object type from vehicle ID
def extract_object_type(id):
    if id.startswith('bike') :
        return 'bicycle'
    elif id.startswith('bus') :
        return 'bus'
    elif id.startswith('moto') :
        return 'motorcycle'
    elif id.startswith('ped') :
       return 'pedestrian'
    elif id.startswith('pt_bus_') :
        return 'bus'
    elif id.startswith('pt_tram_') :
        return 'bus'
    elif id.startswith('truck') :
        return 'truck'
    elif id.startswith('veh') :
        return 'vehicle'
    else:
        return "unknown"
def save_ds(current_ts, cpm_data,detected_objects):
    # Save enhanced dataset
    cpm_dataset=pd.DataFrame(cpm_data)
    cpm_dataset.to_csv(f"burst_adma_with_cpm_multi_sensors/burst_adma_with_cpm_multi_sensors{current_ts}.csv", index=False)
    cpm_dataset=None
    cpm_dataset=pd.DataFrame(detected_objects)
    cpm_dataset.to_csv(f"burst_adma_with_cpm_multi_sensors/burst_adma_with_cpm_multi_sensors{current_ts}_detected_objects.csv", index=False)
    cpm_dataset=None
    print(f"CPM-enhanced dataset with multiple sensors saved as burst_adma_with_cpm_multi_sensors{current_ts}.csv")
  

# Function to simulate CPM detection with multiple sensors
def generate_cpm(vehicles, sensor_types=SENSOR_TYPES, sensor_ranges=SENSOR_RANGES):
    cpm_data = []
    detected_objects = []
    timestamps = vehicles['timestep'].unique()
    
    # Simulate environmental conditions
    weather_conditions = np.random.choice(['clear', 'foggy', 'rainy', 'snowy'], size=len(timestamps))
    visibility_ranges = {
        'clear': 100,
        'foggy': 30,
        'rainy': 50,
        'snowy': 40
    }
    current_ts=0
    for idx, t in tqdm(enumerate(timestamps)):
        current_ts+=1
        if current_ts % MAX_TS==0:
            # Save enhanced dataset
            save_ds(t, cpm_data,detected_objects)
            cpm_data = [] 
            detected_objects=[]
        snapshot = vehicles[vehicles['timestep'] == t]
        positions = snapshot[['x', 'y']].values
        vehicle_ids = snapshot['id'].values
        velocities = snapshot['speed'].values
        headings = snapshot['heading'].values
        accelerations = snapshot['acceleration'].values
        kd_tree = KDTree(positions)
        
        current_weather = weather_conditions[idx]
        visibility_range = visibility_ranges[current_weather]
        
        for i, (vid, pos, vel, head,acceleration) in enumerate(zip(vehicle_ids, positions, velocities, headings,accelerations)):
            object_type = extract_object_type(vid)  # Extract object type from ID
            for sensor in sensor_types:
                sensor_range = min(sensor_ranges[sensor], visibility_range)
                
                # Find nearest neighbors within sensor range
                indices = kd_tree.query_ball_point(pos, sensor_range)
                
                for idx in indices:
                    if idx != i:  # Exclude self
                        detected_objects.append({
                            'cpm_id': f"CPM_{vid}_{t}_{sensor}",
                            'detected_vehicle_id': vehicle_ids[idx],
                            'x_detected': positions[idx][0] + np.random.normal(0, 1),  # Add noise
                            'y_detected': positions[idx][1] + np.random.normal(0, 1),
                            'velocity_detected': velocities[idx] + np.random.normal(0, 0.5),
                            'heading_detected': headings[idx] + np.random.normal(0, 1),
                            'acceleration_detected': accelerations[idx] + np.random.normal(0, 1),
                            'object_type': extract_object_type(vehicle_ids[idx]),
                            'detection_confidence': np.random.uniform(0.6, 1.0)  # Varies with weather
                        })
                
                # Create CPM entry
                cpm_data.append({
                    'cpm_id': f"CPM_{vid}_{t}_{sensor}",
                    'vehicle_id': vid,
                    'timestep': t,
                    'sensor_id': f"Sensor_{vid}_{sensor}",
                    'sensor_type': sensor,
                    'sensor_range': sensor_range,
                    'weather_conditions': current_weather,
                    'visibility_range': visibility_range,
                    'object_type': object_type,
                    #'detected_objects': detected_objects
                })
    if cpm_data:
        # Save enhanced dataset
        save_ds(t, cpm_data,detected_objects)
        cpm_data = [] 
        detected_objects=[]
    # return pd.DataFrame(cpm_data)

# Generate CPM dataset
generate_cpm(vehicles)

# # Save enhanced dataset
# cpm_dataset.to_csv("burst_adma_with_cpm_multi_sensors.csv", index=False)
# print("CPM-enhanced dataset with multiple sensors saved as burst_adma_with_cpm_multi_sensors.csv")
# import sqlite3
# # Create your connection.
# cnx = sqlite3.connect('imported_data/burst_adma_with_cpm_multi_sensors.db')
# cpm_dataset.to_sql(name='burst_adma_with_cpm_multi_sensors', con=cnx)


199it [00:31,  2.47it/s]

CPM-enhanced dataset with multiple sensors saved as burst_adma_with_cpm_multi_sensors199.0.csv


399it [03:42,  1.07s/it]

CPM-enhanced dataset with multiple sensors saved as burst_adma_with_cpm_multi_sensors399.0.csv


599it [10:28,  1.54s/it]

CPM-enhanced dataset with multiple sensors saved as burst_adma_with_cpm_multi_sensors599.0.csv


799it [20:38,  1.86s/it]

CPM-enhanced dataset with multiple sensors saved as burst_adma_with_cpm_multi_sensors799.0.csv


999it [33:41,  2.43s/it] 

: 

5. +Clustering

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
import math
from tqdm import tqdm

# Load BurST-ADMA dataset (Assuming CSV format)
file_path = "data/burst_adma_with_clusters_in_time.csv"  # Update with actual path
data = pd.read_csv(file_path)

# Extract relevant fields
vehicles = data[['id', 'timestep', 'x', 'y', 'speed', 'heading','acceleration','label','cls']]

# Define sensor range (meters)
SENSOR_TYPES = ["LiDAR", "Camera", "Radar"]
SENSOR_RANGES = {"LiDAR": 50, "Camera": 40, "Radar": 80}
MAX_TS=1000
FOLDER_TO_SAVE="burst_adma_with_cpm_multi_sensors_cls"
# Function to extract object type from vehicle ID
def extract_object_type(id):
    if id.startswith('bike') :
        return 'bicycle'
    elif id.startswith('bus') :
        return 'bus'
    elif id.startswith('moto') :
        return 'motorcycle'
    elif id.startswith('ped') :
       return 'pedestrian'
    elif id.startswith('pt_bus_') :
        return 'bus'
    elif id.startswith('pt_tram_') :
        return 'bus'
    elif id.startswith('truck') :
        return 'truck'
    elif id.startswith('veh') :
        return 'vehicle'
    else:
        return "unknown"
def save_ds(current_ts, cpm_data,detected_objects,all_objects):
    # Save cam+cpm dataset
    cpm_dataset=pd.DataFrame(all_objects)
    cpm_dataset.to_csv(f"{FOLDER_TO_SAVE}/burst_adma_with_cpm_multi_sensors{int(current_ts)}_cls_all.csv", index=False)
    cpm_dataset=None
    # Save cam dataset
    cpm_dataset=pd.DataFrame(cpm_data)
    cpm_dataset.to_csv(f"{FOLDER_TO_SAVE}/burst_adma_with_cpm_multi_sensors{int(current_ts)}_cls.csv", index=False)
    cpm_dataset=None 
    # Save cpm dataset
    cpm_dataset=pd.DataFrame(detected_objects)
    cpm_dataset.to_csv(f"{FOLDER_TO_SAVE}/burst_adma_with_cpm_multi_sensors{int(current_ts)}_cls_detected_objects.csv", index=False)
    cpm_dataset=None
    print(f"CPM-enhanced dataset with multiple sensors saved as burst_adma_with_cpm_multi_sensors{int(current_ts)}_cls.csv")
  

# Function to simulate CPM detection with multiple sensors
def generate_cpm(vehicles, sensor_types=SENSOR_TYPES, sensor_ranges=SENSOR_RANGES):
    cpm_data = []
    detected_objects = []
    all_objects = []
    timestamps = vehicles['timestep'].unique()
    
    # Simulate environmental conditions
    weather_conditions = np.random.choice(['clear', 'foggy', 'rainy', 'snowy'], size=len(timestamps))
    visibility_ranges = {
        'clear': 100,
        'foggy': 30,
        'rainy': 50,
        'snowy': 40
    }
    current_ts=0
    for idx, t in tqdm(enumerate(timestamps)):
        if cpm_data and current_ts % MAX_TS==0:
            # Save enhanced dataset
            save_ds(t, cpm_data,detected_objects,all_objects)
            cpm_data = [] 
            detected_objects=[]
            all_objects=[]

            
        current_weather = weather_conditions[idx]
        visibility_range = visibility_ranges[current_weather]

        clasess = vehicles['cls'].unique()
        snapshot_v = vehicles[vehicles['timestep'] == t]
        for idx_cls, t_cls in enumerate(clasess):
            snapshot = snapshot_v[snapshot_v['cls'] == t_cls]
            positions = snapshot[['x', 'y']].values
            vehicle_ids = snapshot['id'].values
            velocities = snapshot['speed'].values
            headings = snapshot['heading'].values
            accelerations = snapshot['acceleration'].values
            labels = snapshot['label'].values
            kd_tree = KDTree(positions)
            
            for i, (vid, pos, vel, head,acceleration,label) in enumerate(zip(vehicle_ids, positions, velocities, headings,accelerations,labels)):
                object_type = extract_object_type(vid)  # Extract object type from ID
                acceleration = 0.0 if math.isnan(acceleration) else acceleration
                # add self
                cam = {
                    'timestep': t,
                    'cpm_id': f"CPM_{vid}_{t}",
                    'vehicle_id': vid,
                    'object_type': object_type,
                    'cls':t_cls,
                    'label': label , 
                    'x': pos[0] ,
                    'y': pos[1] ,
                    'speed': vel,
                    'heading': head,
                    'acceleration': acceleration,
                    'is_self': 1
                }
                cpm_data.append(cam)
                all_objects.append({
                                'timestep': cam['timestep'],
                                'cpm_id':   cam['cpm_id'],
                                'vehicle_id': cam['vehicle_id'],
                                'detected_vehicle_id': None,
                                'object_type':  cam['object_type'],
                                'detected_object_type':  None,
                                'cls_detected': cam['cls'],
                                'label_detected':  cam['label'],
                                'x_detected': cam['x'],
                                'y_detected': cam['y'],
                                'speed_detected':  cam['speed'],
                                'heading_detected':  cam['heading'],
                                'acceleration_detected':  cam['acceleration'],
                                'is_self':  cam['is_self'],
                                'sensor_id': None,
                                'sensor_type': None,
                                'sensor_range': None,
                                'weather_conditions': current_weather,
                                'visibility_range': visibility_range,
                                'detection_confidence': None
                            })
                for sensor in sensor_types:
                    sensor_range = min(sensor_ranges[sensor], visibility_range)
                    
                    # Find nearest neighbors within sensor range
                    indices = kd_tree.query_ball_point(pos, sensor_range)
                    
                    for idx in indices:
                        if idx != i:  # Exclude self
                            # Create CPM entry
                            cpm = {
                                'timestep': t,
                                'cpm_id': f"CPM_{vid}_{t}",
                                'vehicle_id': vid,
                                'detected_vehicle_id': vehicle_ids[idx],
                                'object_type': object_type,
                                'detected_object_type': extract_object_type(vehicle_ids[idx]),
                                'cls_detected':t_cls, #no noise? TODO:
                                'label_detected': labels[idx] , #no noise? TODO:
                                'x_detected': positions[idx][0] + np.random.normal(0, 1),  # Add noise
                                'y_detected': positions[idx][1] + np.random.normal(0, 1),
                                'speed_detected': velocities[idx] + np.random.normal(0, 0.5),
                                'heading_detected': headings[idx] + np.random.normal(0, 1),
                                'acceleration_detected': (0.0 if math.isnan(accelerations[idx]) else accelerations[idx]) + np.random.normal(0, 1),
                                'is_self': 0,
                                'sensor_id': f"Sensor_{vid}_{sensor}",
                                'sensor_type': sensor,
                                'sensor_range': sensor_range,
                                'weather_conditions': current_weather,
                                'visibility_range': visibility_range,
                                'detection_confidence': np.random.uniform(0.6, 1.0),  # Varies with weather
                            }
                            detected_objects.append(cpm)
                            all_objects.append(cpm)
                    
                # cpm_data.append(cam)
        current_ts+=1

    if cpm_data:
        # Save enhanced dataset
        save_ds(t, cpm_data,detected_objects,all_objects)
        cpm_data = [] 
        detected_objects=[]
        all_objects=[]
    # return pd.DataFrame(cpm_data)

# Generate CPM dataset
generate_cpm(vehicles)

# # Save enhanced dataset
# cpm_dataset.to_csv("burst_adma_with_cpm_multi_sensors.csv", index=False)
# print("CPM-enhanced dataset with multiple sensors saved as burst_adma_with_cpm_multi_sensors.csv")
# import sqlite3
# # Create your connection.
# cnx = sqlite3.connect('imported_data/burst_adma_with_cpm_multi_sensors.db')
# cpm_dataset.to_sql(name='burst_adma_with_cpm_multi_sensors', con=cnx)


1000it [03:27,  4.82it/s]


CPM-enhanced dataset with multiple sensors saved as burst_adma_with_cpm_multi_sensors999_cls.csv


# Real

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
import math
from tqdm import tqdm

# Load BurST-ADMA dataset (Assuming CSV format)
file_path = "data/burst_adma_with_clusters_in_time.csv"  # Update with actual path
data = pd.read_csv(file_path)

# Extract relevant fields
vehicles = data[['id', 'timestep', 'x', 'y', 'speed', 'heading','acceleration','label','cls']]

# Define sensor range (meters)
SENSOR_TYPES = ["LiDAR", "Camera", "Radar"]
SENSOR_RANGES = {"LiDAR": 50, "Camera": 40, "Radar": 80}
MAX_TS=1000
FOLDER_TO_SAVE=f"burst_adma_with_cpm_multi_sensors_cls_lidar_{SENSOR_RANGES['LiDAR']}"
# Function to extract object type from vehicle ID
def extract_object_type(id):
    if id.startswith('bike') :
        return 'bicycle'
    elif id.startswith('bus') :
        return 'bus'
    elif id.startswith('moto') :
        return 'motorcycle'
    elif id.startswith('ped') :
       return 'pedestrian'
    elif id.startswith('pt_bus_') :
        return 'bus'
    elif id.startswith('pt_tram_') :
        return 'bus'
    elif id.startswith('truck') :
        return 'truck'
    elif id.startswith('veh') :
        return 'vehicle'
    else:
        return "unknown"
def save_ds(current_ts,all_objects):
    # Save cam+cpm dataset
    cpm_dataset=pd.DataFrame(all_objects)
    cpm_dataset.to_csv(f"{FOLDER_TO_SAVE}/burst_adma_with_cpm_multi_sensors{int(current_ts)}_cls_all.csv", index=False)
    cpm_dataset=None 
    print(f"CPM-enhanced dataset with multiple sensors saved as burst_adma_with_cpm_multi_sensors{int(current_ts)}_cls.csv")
  

# Function to simulate CPM detection with multiple sensors
def generate_cpm(vehicles, sensor_types=SENSOR_TYPES, sensor_ranges=SENSOR_RANGES):
    all_objects = []
    timestamps = vehicles['timestep'].unique()
    
    # Simulate environmental conditions
    weather_conditions = np.random.choice(['clear', 'foggy', 'rainy', 'snowy'], size=len(timestamps))
    visibility_ranges = {
        'clear': 100,
        'foggy': 30,
        'rainy': 50,
        'snowy': 40,
        #TODO'sun':?
    }
    current_ts=0
    for idx, t in tqdm(enumerate(timestamps)):
        if all_objects and current_ts % MAX_TS==0:
            # Save enhanced dataset
            save_ds(t,  all_objects) 
            all_objects=[]

            
        current_weather = weather_conditions[idx]
        visibility_range = visibility_ranges[current_weather]

        clasess = vehicles['cls'].unique()
        snapshot_v = vehicles[vehicles['timestep'] == t]
        for idx_cls, t_cls in enumerate(clasess):
            snapshot = snapshot_v[snapshot_v['cls'] == t_cls]
            positions = snapshot[['x', 'y']].values
            vehicle_ids = snapshot['id'].values
            velocities = snapshot['speed'].values
            headings = snapshot['heading'].values
            accelerations = snapshot['acceleration'].values
            labels = snapshot['label'].values
            kd_tree = KDTree(positions)
            
            for i, (vid, pos, vel, head,acceleration,label) in enumerate(zip(vehicle_ids, positions, velocities, headings,accelerations,labels)):
                object_type = extract_object_type(vid)  # Extract object type from ID
                acceleration = 0.0 if math.isnan(acceleration) else acceleration
                # add self
                cam = {
                    'timestep': t,
                    'cpm_id': f"CPM_{vid}_{t}",
                    'vehicle_id': vid,
                    'object_type': object_type,
                    'cls':t_cls,
                    'label': label , 
                    'x': pos[0] ,
                    'y': pos[1] ,
                    'speed': vel,
                    'heading': head,
                    'acceleration': acceleration,
                    'is_self': 1
                } 
                all_objects.append({
                                'timestep': cam['timestep'],
                                'cpm_id':   cam['cpm_id'],
                                'vehicle_id': cam['vehicle_id'],
                                'detected_vehicle_id': '',
                                'object_type':  cam['object_type'],
                                'detected_object_type':  '',
                                'cls_detected': cam['cls'],
                                'label_detected':  cam['label'],
                                'x_detected': cam['x'],
                                'y_detected': cam['y'],
                                'speed_detected':  cam['speed'],
                                'heading_detected':  cam['heading'],
                                'acceleration_detected':  cam['acceleration'],
                                'is_self':  cam['is_self'],
                                'sensor_id': '',
                                'sensor_type': '',
                                'sensor_range': '0',
                                'weather_conditions': current_weather,
                                'visibility_range': visibility_range,
                                'detection_confidence': '0'
                            })
                for sensor in sensor_types:
                    sensor_range = min(sensor_ranges[sensor], visibility_range)
                    
                    # Find nearest neighbors within sensor range
                    indices = kd_tree.query_ball_point(pos, sensor_range)
                    
                    for idx in indices:
                        if idx != i:  # Exclude self
                            # Create CPM entry
                            cpm = {
                                'timestep': t,
                                'cpm_id': f"CPM_{vid}_{t}",
                                'vehicle_id': vid,
                                'detected_vehicle_id': vehicle_ids[idx],
                                'object_type': object_type,
                                'detected_object_type': extract_object_type(vehicle_ids[idx]),
                                'cls_detected':t_cls, #no noise? TODO:
                                'label_detected': labels[idx] , #no noise? TODO:
                                'x_detected': positions[idx][0] + np.random.normal(0, 1),  # Add noise
                                'y_detected': positions[idx][1] + np.random.normal(0, 1),
                                'speed_detected': velocities[idx] + np.random.normal(0, 0.5),
                                'heading_detected': headings[idx] + np.random.normal(0, 1),
                                'acceleration_detected': (0.0 if math.isnan(accelerations[idx]) else accelerations[idx]) + np.random.normal(0, 1),
                                'is_self': 0,
                                'sensor_id': f"Sensor_{vid}_{sensor}",
                                'sensor_type': sensor,
                                'sensor_range': sensor_range,
                                'weather_conditions': current_weather,
                                'visibility_range': visibility_range,
                                'detection_confidence': np.random.uniform(0.6, 1.0),  # Varies with weather
                            } 
                            all_objects.append(cpm)
                     
        current_ts+=1

    if all_objects:
        # Save enhanced dataset
        save_ds(t,  all_objects) 
        all_objects=[] 

# Generate CPM dataset
generate_cpm(vehicles)
 


FileNotFoundError: [Errno 2] No such file or directory: 'data/burst_adma_with_clusters_in_time.csv'